<h1>Algorithms tests</h1>

In [74]:
import sys
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [75]:
sys.path.append('/Users/Evgeny/wonderful_programming/fuzzy-fusion-venv/fuzzy-fusion/src/')
from generator.generator import generator
from algorithm.gibbs import gibbs_sampl
from algorithm.gibbs_fuzzy import gibbs_fuzzy
from algorithm.em import em

print 'Python version ' + sys.version
print 'Pandas version ' + pd.__version__

Python version 2.7.10 (default, Oct 23 2015, 18:05:06) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.0.59.5)]
Pandas version 0.17.1


In [77]:
s_number = 5
obj_number = 20
cl_size = 15
cov_list = [0.7]*s_number
p_list = [0.7]*s_number
accuracy_list = [0.8]*s_number

accuracy_for_df = [[i, accuracy_list[i]] for i in range(s_number)]
accuracy_data = pd.DataFrame(accuracy_for_df, columns=['S', 'A'])

result_list = []
for g_true in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    
    print g_true
    print '*****'
        
    for i in range(10):
        ground_truth = [random.randint(0, 1) for i in range(obj_number)]
        data, g_data = generator(cov_list, p_list, ground_truth, cl_size, g_true)
        em_d = em(data=data, accuracy=accuracy_data, truth_obj_list=ground_truth)[0]
        while True:
            try:
                g_d = gibbs_sampl(data=data, accuracy_data=accuracy_data, truth_obj_list=ground_truth)[0]
                gf_d = gibbs_fuzzy(data=data, accuracy_data=accuracy_data, g_data=g_data, truth_obj_list=ground_truth)[0]
                break
            except ZeroDivisionError:
                print 'zero {}'.fprmat(i)
        result_list.append([g_true, em_d, g_d, gf_d])

0.0
*****
0.1
*****
!
0.2
*****
!
0.3
*****
0.4
*****
!
0.5
*****
0.6
*****
!
!
0.7
*****
!
0.8
*****
0.9
*****
!
1.0
*****
!


In [78]:
columns = ['p_g', 'em', 'g', 'gf']
result_data = pd.DataFrame(data=result_list, columns=columns)
result_data.to_csv('output_dat15.csv')

In [79]:
x = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
em, em_std = [], []
g, g_std = [], []
gf, gf_std = [], []
for g_true in x:
    data = result_data[result_data.p_g == g_true]
    em.append(np.mean(data.em))
    em_std.append(np.std(data.em))
    g.append(np.mean(data.g))
    g_std.append(np.std(data.g))
    gf.append(np.mean(data.gf))
    gf_std.append(np.std(data.gf))

In [80]:
f, axarr = plt.subplots()
f.set_size_inches(14, 5)
axarr.plot(x, em, 'r', label='em')
axarr.errorbar(x, em, yerr=em_std, fmt='r')
axarr.plot(x, g, 'g', label='gibbs')
axarr.errorbar(x, g, yerr=g_std, fmt='g')
axarr.plot(x, gf, 'b', label='gibbs fuzzy')
axarr.errorbar(x, gf, yerr=gf_std, fmt='b')
axarr.set_xlabel('Probability of g=true_obj', fontsize=14)
axarr.set_ylabel('Dist', fontsize=14)
axarr.legend(loc='lower center', fancybox=True, shadow=True, ncol=3)
axarr.set_title('Dist to truth, cluster_size={}'.format(cl_size), fontsize=15, fontweight='bold')
axarr.axis([-0.1, 1.1, 0.4, 0.9])
plt.show()